In [12]:
""" SOBRE:

ESTE EXTRATOR FOI DESENVOLVIDO NO FIM DE 2019 POR ALESSANDRO DE ALMEIDA SÍCOLI PARA A ESTRUTURAÇÃO DE DADOS
PROVENIENTES DE NOTAS FISCAIS DE SERVIÇO PARA O BANCO DO BRASIL. 
COMO AS NOTAS FISCAIS DE SERVIÇO TEM SEU LAYOUT DIFERENTE PARA CADA CIDADE DO PAÍS, O RECEBIMENTO E CADASTRO DOS
DADOS DAS NOTAS FISCAIS NÃO ERA PASSÍVEL DE AUTOMAÇÃO. NO ENTANTO, COM A IDENTIFICAÇÃO DE PADRÕES COMUNS E UM POUCO
DE CRIATIVIDADE, FOI POSSÍVEL, COM A UTILIZAÇÃO DE TÉCNICAS DE PROCESSAMENTO DE LINGUAGEM NATURAL, COMO O REGEX E
FERRAMENTAS DE ANÁLISE E ORGANIZAÇÃO DE DADOS COMO O PANDAS, RESOLVER, COM REDUÇÃO SIGNIFICATIVA DE LINHAS DE CÓDIGO,
UM PROBLEMA QUE LEVAVA AO DESPERDÍCIO DE MILHARES DE HORAS DE TRABALHO/MÊS NO SETOR DE ADMINISTRAÇÃO DE CONTRATOS DA
EMPRESA, DADO O VOLUME EXPRESSIVO DE DOCUMENTOS PROCESSADOS PARA TODOS OS CONTRATOS DE SERVIÇO DO BANCO.
  

"""




from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import resolve1

file_path = 'C:\\Users\\f0427837\\Desktop\\teste-NF.pdf'
fp = open(file_path, 'rb')

#O CÓDIGO ABAIXO PERMITE FAZER A CONTAGEM DO NÚMERO DE PÁGINAS PARA POSTERIOR ITERAÇÃO
parser = PDFParser(fp)
document = PDFDocument(parser)
pages=resolve1(document.catalog['Pages'])['Count']
#print(pages) #CASO SE QUEIRA CONFERIR O NÚMERO DE PÁGINAS


#ESTABELECENDO OS PARÂMETROS DO PDFMINER PARA CAPTURA DO TEXTO NO PDF
rsrcmgr = PDFResourceManager()
retstr = StringIO()
codec = 'utf-8' # IMPORTANTE PARA EVITAR PROBLEMAS NA REPRODUÇÃO DOS CARACTERES
laparams = LAParams()
device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
password = ""
maxpages = 0
caching = True
pagenos=set()



#Cria-se uma lista vazia
lista = []
#CRIAMOS UMA VARIÁVEL PAGENUMBER, COMEÇANDO POR 0 PARA ITERAR ABAIXO
pn = 0
#Fazemos uma iteração em casos de documentos com mais de uma página

for pagenumber, page in enumerate(PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True)):
    #Enquanto pagenumber existir e for igual pn, captura-se os dados e cria-se uma vatiável nf+string página
    if pagenumber == pn:
        interpreter.process_page(page)
        x = retstr.getvalue()
        #Usa-se o globals() para criar o nome de uma variável baseada em outra variável
        globals()['nota_corretagem_pg_'+str(pagenumber+1)] = x
        var =  'nota_corretagem_pg_'+str(pagenumber+1)
        #Criamos uma lista com o nome das variáveis que pode ser útil para verificações
        lista.append(var)
        x= ""
        retstr.truncate(0)
        retstr.seek(0)
    pn += 1

fp.close()
device.close()
retstr.close()



In [21]:
texto = globals()["nf1")
print (texto)

 [Número ds Nota

PREFEITURA DO MUNICÍPIO DE SÃO PAULO

SECRETARIA MUNICIPAL DA FAZENDA

a

NOTAFISCAL ELETRÔNICA DE SERVIÇOS -NFS-e

"oo?
o Hora do Emissão

Baia
09/05/2019 16:28:06
 [cosgodeVencação

BFPB-LEBI

 

 

À),
=D)

Nomefent Soc: GANTRAES CENTRALLDA VE
Endereço R SILVEIRA MARTINS00115, CONJ 21 - SE- CEP: 01019-000

ESCENTRAL Município: São Paulo

ur se

PRESTADORDE SERVIÇOS

 

TOMADORDE SERVIÇOS.

 

vESP

Email —

Insenção Municipal —

Noreiazão Social

NomeiRazãsSocial BANCO DO BRASIL S.A.
CPFICNP: 00,000.000/5T60.02
Endereço: AV JOAQUIM AFONSO DA COSTA263 - BELA VISTA - CEP: 14820.000
   
INTERMEDIÁRIO DE SERVIÇOS
coros —
FORNECIMENTO DE MÃO DE OBRA DE LIMPESA DECORRENTE DO CONTRATO 2017.1421.0394, REFERENTE AO
PERÍODO DE 01/04/2019 A 30/04/2019.
vão DE OBRA:
R$
RET. INSS:
VALE TRANSPORTE:
VALE ALIMENTAÇÃO:
INSUMOS (MATERIAL DE LIMPEZA, EQUIPAMENTOS E SACOS DE LIXOS):
as:
Cie:

DISCRIMINAÇÃO DOS SERVIÇOS

1.580,4
R$147,00

0207
sooo2-4 

(R$173,05)

R$ 82,20

 

 
 
  


In [22]:
import re #IMPORTA A BIBLIOTECA PARA USO DE REGEX
import pandas as pd #IMPORTA A BIBLIOTECA DE TRATAMENTO DE DF PANDAS

"""Criamos uma lista vazia para cada elemento da nota fiscal que 
gostariamos que fosse capturado. Dessa forma, posteriormente, poderemos
utilizar esses dados para montar um dicionário e finalmente, criar
um dataframe com pandas com base nos dados dos dicionários."""

Nota = []
data_emissao = []
cod_ver = []
CNPJ_em = []
empresa = []
Mun_em = []
UF_em = []
CNPJ_tom = []
Mun_tom = []
UF_tom = []
val_NF = []

for n in lista:
    """ IMPORTANTE!!!!
    Lembrando que cada n contém todo o texto da nota
    COMO FORAM SALVAS CADA VARIÁVEL CONTENDO O TEXTO DE CADA PAGINA EM UMA LISTA
    NO FORMATO DE STRING, CRIAMOS A VARIÁVEL TEXTO, USANDO A FUNÇÃO GLOBALS()
    PARA CONSEGUIR LER CADA N STRING DA LISTA COMO UMA VARIÁVEL CONTENDO O
    TEXTO DA NOTA FISCAL, ITERANDO A STRING COMO SE FOSSE UMA ITERAÇÃO DE
    VARIÁVEIS"""
    
    texto = globals()[n]
          
    split1 = texto.split("Número da Nota")[1].split()
    Nota.append([i for i in split1 if re.search(r'\d+', i)][0])
    
    #No  re.findall()cria uma lista com n elementos que se encontram
    # no texto que satisfação o criterio de busca daquela expressão regular
    # sendo assim, para usarmos a função split temos que acessar a string 
    #dentro da lista, usando [n] para encontrar o elemento de posição n
    #naquela lista
    
    """A Função split() nos retorna uma lista com cada termo de um texto
    Na funcão split() se usarmos split(argumento)[0], irá retornar todo
    o texto antes do argumento encontrado, se colocar [1] pega o texto após
    a primeira ocorrência até a segunda se houver, se colocar [2] irá pegar
    o texto até após a segunda ocorrência até a terceira se houver...
    Lembrando, se usar o [] ele não retorna lista mas o texto até determinado
    ponto, então para converter em lista deve-se utilizar o .split() no final
    Se não usarmos[] ele irá apenas retirar apenas aquele termo da lista
    usamos o .split() no final da função para separar cada palavra em um
    elemento na lista"""
    
    a = texto.split(re.findall(r'(?i)emissão', texto)[0])[1].split()
    data_emissao.append([i for i in a if re.search(r'\d+', i)][0])
    
    b =  texto.split(re.findall(r'(?i)verificação', texto)[0])[1].split()
    cod_ver.append([i for i in b if re.search(r'\d+', i)][0])
    
    c =  texto.split(re.findall(r'(?i)cnpj', texto)[0])[1].split()
    CNPJ_em.append([i for i in c if re.search(r'\d+', i)][0])
    
    
    #Nesse não vamos usar split no final pois iremos pegar mais
    # de uma palavra, iremos separar por linha '\n' e pegarmos o
    #nome da empresa todo
    d =  texto.split(re.findall(r'(?i)Social:', texto)[0])[1]
    #Após pegar o texto, Vamos splitar em uma listacom base em linhas \n
    split_empresa = re.split("\n+", d)
    #Vamos pegar o primeiro termos de uma lista que nao seja em branco '\S+'
    empresa.append([i for i in split_empresa if re.search(r'\S+', i)][0])
    
    e =  texto.split(re.findall(r'(?i)pio:', texto)[0])[1]
    #Após pegar o texto, Vamos splitar em uma listacom base em linhas \n
    split_mun_em = re.split("\n+", e)
    #Vamos pegar o primeiro termos de uma lista que nao seja em branco '\S+'
    Mun_em.append([i for i in split_mun_em if re.search(r'\S+', i)][0])
    
    f = texto.split(re.findall(r'(?i)uf:', texto)[0])[1].split()[0]
    UF_em.append(f)
    
    #REPARE QUE AO CONTRÁRIO DO PRIMEIRO CNPJ, IREMOS SPLITAR O TEXTO
    #APÓS A SEGUNDA [2] OCORRÊNCIA QUE CONTIVER O TERMO "CNPJ"
    g =  texto.split(re.findall(r'(?i)cnpj', texto)[0])[2]
    #Após pegar o texto, Vamos splitar em uma listacom base em linhas \n
    split_cnpj_tom = re.split("\n+", g)
    #Vamos pegar o primeiro termoda lista que contenha numeros
    CNPJ_tom.append([i for i in split_cnpj_tom if re.search(r'\d+', i)][0])
    
    h =  texto.split(re.findall(r'(?i)pio:', texto)[0])[2]
    #Após pegar o texto, Vamos splitar em uma listacom base em linhas \n
    split_mun_tom = re.split("\n+", h)
    #Vamos pegar o primeiro termos de uma lista que nao seja em branco '\S+'
    Mun_tom.append([i for i in split_mun_tom if re.search(r'\S+', i)][0])
    
    j = texto.split(re.findall(r'(?i)uf:', texto)[0])[2].split()[0]
    UF_tom.append(j)
    
    #Pegamos o primeiro item contendo número após o termo "valor total":
    k = texto.split(re.findall(r'(?i)valor total', texto)[0])[1].split()
    val_NF.append([i for i in k if re.search(r'\d+', i)][0])
    
    
"""Criamos um dicionário com as variáveis para depois convertermos o dict
com as listas em um DataFrame no Pandas"""

d = {'Empresa' : empresa, 'CNPJ do Emissor': CNPJ_em,'Município Emissor': Mun_em,
     'UF Emissor' : UF_em, 'CNPJ Tomador' : CNPJ_tom, 'Município Tomador' : Mun_tom, 
     'UF Tomador' : UF_tom, 'Número da NF': Nota, 'Data de Emissão': data_emissao, 
     'Valor NF' : val_NF, 'Código de Verificação' : cod_ver
    }


#Agora Criamos um Pandas Data Frame

df = pd.DataFrame(d)

df.head()

,Empresa,CNPJ do Emissor,Município Emissor,UF Emissor,CNPJ Tomador,Município Tomador,UF Tomador,Número da NF,Data de Emissão,Valor NF,Código de Verificação
0,RS CONSULTORIA E SERVICOS DE GESTAO EMPRESARIA...,06.350.648/0001-74,Lucélia,SP,00.000.000/2940-80,São Paulo,SP,154661,08/01/2019,"10.071,84",KAHRKK-154661/2019
1,RS CONSULTORIA E SERVICOS DE GESTAO EMPRESARIA...,06.350.648/0001-74,Lucélia,SP,00.000.000/1174-63,Praia Grande,SP,154662,08/01/2019,"6.142,51",RAHKRR-154662/2019


In [3]:
split6 =  nf1.split(re.findall(r'(?i)pio:', nf1)[0])[2]
#Após pegar o texto, Vamos splitar em uma listacom base em linhas \n
split_mun_em = re.split("\n+", split6)
#Vamos pegar o primeiro termos de uma lista que nao seja em branco '\S+'
Mun_Em = [i for i in split_mun_em if re.search(r'\S+', i)][0]
Mun_Em = Mun_Em.upper()    
print(Mun_Em)

  SÃO PAULO


In [4]:
s =  globals()[n].split(re.findall(r'(?i)uf:', globals()[n])[0])[1]
#Após pegar o texto, Vamos splitar em uma listacom base em linhas \n
s_uf_em = re.split("\n+", s)
#Vamos pegar o primeiro termos de uma lista que nao seja em branco '\S+'
S_uf_em = [] 
S_uf_em.append([i for i in s_uf_em if re.search(r'\S+', i)][0])
print(S_uf_em)    

['SP']


In [5]:
a = globals()[n]
print(a)


PREFEITURA DO MUNICÍPIO DE LUCELIA

SECRETÁRIA DE TRIBUTAÇÃO

NOTA FISCAL DE SERVIÇO ELETRÔNICA - NFe

RPS No. 155575  Série: ELETRÔNICA

PRESTADOR DE SERVIÇOS

Número da Nota

Data do Serviço

154662

08/01/2019

Data e Hora de Emissão

08/01/2019 11:24:00

Código de Verificação

RAHKRR-154662/2019

CNPJ/CPF: 

06.350.648/0001-74

Inscrição Municipal:

324400

Inscrição Estadual:

 ISENTO 

Nome/Razão Social: 

RS CONSULTORIA E SERVICOS DE GESTAO EMPRESARIAL LTDA

Telefone:

(18) 3551-9999

Endereço: 

RUA MANOEL LOPES, 1857, CENTRO - CEP:17780-000

Município: 

Lucélia

CNPJ/CPF: 

00.000.000/1174-63

Nome/Razão Social: 

BANCO DO BRASIL S.A.

TOMADOR DE SERVIÇOS

Insc.Municipal:

UF: 

SP

Insc. Estadual::

Endereço:  PRES. COSTA E SILVA, 963. _. BOQUEIRAO. CEP: 11701-902

Município:  Praia Grande

UF:  SP

E-MAIL:

Telefone:

PRESTACAO DE SERVICOS DE LIMPEZA E CONSERVACAO - PUBLICO REF. AO MES DE DEZEMBRO/2018 COM 
FORNECIMENTO DE INSUMOS CONFORME DOCUMENTO N02 DO CONTRATO NO VALOR

In [6]:
split6 = testenota.split(re.findall(r'(?i)cnpj', testenota)[0])[2].split()
CNPJ_tom = [i for i in split6 if re.search(r'\d+', i)][0]
print(CNPJ_tom)

NameError: name 'testenota' is not defined

In [9]:
for n in lista:
    texto = globals()[n]
    
    b = texto.split(re.findall(r'(?i)uf:', texto)[0])[0].split()
    b1 = [i for i in b if re.search(r'\S+', i)][0]
    
    c =  a.split(re.findall(r'(?i)emissão', a)[0])[1].split()
    c1 = [i for i in c if re.search(r'\d+', i)][0]
    

    print (b1)



PREFEITURA
PREFEITURA


In [20]:
k = texto.split(re.findall(r'(?i)da nota', texto)[1])[1].split()
k1 = [i for i in k if re.search(r'\d+', i)][0]
print(k1)

6.142,51
